In [2]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# %pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


/usr/local/miniconda3/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train = pd.read_csv(r'train.csv', encoding='utf_8_sig')
train.fillna('', inplace=True)
test = pd.read_csv(r'test.csv', encoding='utf_8_sig')
test.fillna('', inplace=True)

In [4]:
train

,label,text,diff
0,positive,XSS 취약점 수정,diff --git a/adm/board_form_update.php b/adm/b...
1,negative,"Auto merge of #4754 - daira:fix-enable-debug, ...",diff --git a/depends/hosts/darwin.mk b/depends...
2,positive,CAMEL-9309: Make it easier to turn on|off java...,diff --git a/components/camel-ahc/src/main/jav...
3,positive,Fixed bug #72627: Memory Leakage In exif_proce...,diff --git a/ext/exif/exif.c b/ext/exif/exif.c...
4,negative,Merge branch 'dev',diff --git a/.htaccess b/.htaccess\nindex 3293...
...,...,...,...
7073,positive,Restrict permissions on cinder configuration\n...,diff --git a/manifests/base.pp b/manifests/bas...
7074,positive,avformat/avidec: Limit formats in gab2 to srt ...,diff --git a/libavformat/avidec.c b/libavforma...
7075,positive,Fixed possible direct calls,diff --git a/phpmyfaq/admin/stat.ratings.php b...
7076,positive,"Reject qname's wirelength > 255, `chopOff()` h...",diff --git a/pdns/dnsparser.cc b/pdns/dnsparse...


In [5]:
train_code_change = list(train['diff'].astype(str))
test_code_change = list(test['diff'].astype(str))

In [6]:
# !pip install setfit
from commitfit import get_templated_dataset,sample_dataset
from datasets import Dataset, load_metric

In [7]:
Dataset_train = Dataset.from_pandas(train)
test_dataset = Dataset.from_pandas(test)

In [8]:
train_dataset = get_templated_dataset(Dataset_train, candidate_labels=['positive','negative'], sample_size=8)

In [9]:
# list(train['message'].astype(str).values)
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 7094
})

In [10]:
len(train)

7078

In [11]:
train['label'].value_counts()

label
negative    4443
positive    2635
Name: count, dtype: int64

In [12]:
test['label'].value_counts()

label
negative    1904
positive    1130
Name: count, dtype: int64

In [13]:
len(train_dataset)

7094

In [14]:
from sklearn import  metrics

def compute_metrics(y_pred, y_test):
    # print(y_pred,y_test)
    # classification_report = metrics.classification_report(y_test,y_pred)
    accuracy_score = metrics.accuracy_score(y_test,y_pred)
    precision_score = metrics.precision_score(y_test,y_pred, average='weighted')
    recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
    f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

    # return {"classification_report": classification_report}
    return {"accuracy": accuracy_score, "precision": precision_score,"recall": recall_score, "f1":f1_score}

In [15]:
test['label'].value_counts()

label
negative    1904
positive    1130
Name: count, dtype: int64

In [16]:
train_dataset

Dataset({
    features: ['label', 'text', 'diff'],
    num_rows: 7094
})

In [17]:
train

,label,text,diff
0,positive,XSS 취약점 수정,diff --git a/adm/board_form_update.php b/adm/b...
1,negative,"Auto merge of #4754 - daira:fix-enable-debug, ...",diff --git a/depends/hosts/darwin.mk b/depends...
2,positive,CAMEL-9309: Make it easier to turn on|off java...,diff --git a/components/camel-ahc/src/main/jav...
3,positive,Fixed bug #72627: Memory Leakage In exif_proce...,diff --git a/ext/exif/exif.c b/ext/exif/exif.c...
4,negative,Merge branch 'dev',diff --git a/.htaccess b/.htaccess\nindex 3293...
...,...,...,...
7073,positive,Restrict permissions on cinder configuration\n...,diff --git a/manifests/base.pp b/manifests/bas...
7074,positive,avformat/avidec: Limit formats in gab2 to srt ...,diff --git a/libavformat/avidec.c b/libavforma...
7075,positive,Fixed possible direct calls,diff --git a/phpmyfaq/admin/stat.ratings.php b...
7076,positive,"Reject qname's wirelength > 255, `chopOff()` h...",diff --git a/pdns/dnsparser.cc b/pdns/dnsparse...


In [18]:
model_id = "sentence-transformers/paraphrase-mpnet-base-v2"

In [19]:
from commitfit import CommitFitModel
model = CommitFitModel.from_pretrained(model_id)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [20]:
from commitfit import CommitFitTrainer

trainer = CommitFitTrainer(
    model=model,
    train_dataset=train_dataset,
    train_code_change = train_code_change,
    test_code_change = test_code_change,
    eval_dataset=test_dataset,
    metric = compute_metrics,
    num_iterations=20,
    num_epochs=1
)

In [21]:
%%time
trainer.train()
fewshot_metrics = trainer.evaluate()
fewshot_metrics

Generating Training Pairs: 100%|██████████| 20/20 [00:18<00:00,  1.11it/s]
***** Running training *****
  Num examples = 283760
  Num epochs = 1
  Total optimization steps = 17735
  Total train batch size = 16
Iteration:   2%|▏         | 346/17735 [01:42<1:24:06,  3.45it/s]

Iteration:   8%|▊         | 1389/17735 [06:57<1:20:18,  3.39it/s]

Iteration:  16%|█▌        | 2762/17735 [13:53<1:26:20,  2.89it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)


Iteration: 100%|█████████▉| 17684/17735 [1:28:47<00:16,  3.10it/s]
/usr/local/miniconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the n

CPU times: user 2h 25min 49s, sys: 3min 58s, total: 2h 29min 48s
Wall time: 1h 33min 20s


{'accuracy': 0.9001318391562294,
 'precision': 0.9007466686844912,
 'recall': 0.9001318391562294,
 'f1': 0.9003647762088848}

In [23]:
fewshot_metrics

{'accuracy': 0.9001318391562294,
 'precision': 0.9007466686844912,
 'recall': 0.9001318391562294,
 'f1': 0.9003647762088848}